In [ ]:
!pip install pytesseract

In [ ]:
import pytesseract
import os
import pandas as pd
from PIL import Image


In [ ]:

# Path to the image you want to extract text from
path = os.path.join(os.getcwd(), 'noisy_invoice.png')
image = Image.open(path).convert("RGB")

# show image
image


In [ ]:

# Optional: Apply image processing to clean up the image
# and improve OCR accuracy. For example, thresholding:
# image = image.convert('L')  # Convert image to grayscale
# threshold = 200
# image = image.point(lambda p: p > threshold and 255)

# Use Tesseract to do OCR on the image
text = pytesseract.image_to_string(image)

# Print the text
print(text)


In [ ]:
def grab_raw_data(image):
    # Use pytesseract to get the data
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    # Initialize the response list
    response = []

    # Loop over each item in the 'text' part of the data
    for i in range(len(data['text'])):
        if data['text'][i].strip() != '':  # This filters out empty text results
            x1, y1, width, height = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            x2, y2 = x1 + width, y1 + height
            
            # Here we don't have font information, so we'll omit that.
            # Pytesseract does not extract font family or size information.
            entry = {
                'index': i,
                'position': [x1, y1, x2, y2],
                'text': data['text'][i]
            }
            response.append(entry)

    # Now 'response' will be a list of dictionaries with the text and its bounding box coordinates.
    return response

response = grab_raw_data(image)
response


In [ ]:
print('\n'.join(r['text'] for r in response))

In [ ]:

df = pd.DataFrame(response)
df

In [ ]:
from PIL import ImageDraw
def draw_boxes(image, response):
    """Draws boxes around text on the image"""
    draw = ImageDraw.Draw(image)
    for item in response:
        draw.rectangle(item['position'], outline='red')
    return image



In [ ]:
draw_boxes(image, response)